## Experimenting with the library

In [ ]:
import os 

In [ ]:
os.chdir("../")

In [ ]:
print(os.getcwd())
musicPath = os.getcwd()

In [ ]:
os.chdir(musicPath+"/code/midicsv/")

In [ ]:
import subprocess as sp

In [ ]:
outputFile =open(musicPath+'/output/test.csv','w+')
errorFile =open('test.csv','w+')

In [ ]:
print(os.getcwd())
os.listdir()

In [ ]:
sp.call(['./midicsv', '-v', musicPath+"/data/maestro/2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi"], stdout=outputFile, stderr=errorFile)
outputFile.close()
errorFile.close()

## Feed into Pandas DF

In [1]:
import os 
import subprocess
import sys
import pandas as pd
import math
import concurrent.futures
import numpy as np
os.chdir("../")
musicPath = os.getcwd()
dataPath = musicPath + "/data"

os.chdir(musicPath+"/code/midicsv/")

In [2]:
def midiToCsv(restofPath):
    cmd = ['./midicsv', '-v', musicPath+restofPath]
    a = subprocess.Popen(cmd, stdout=subprocess.PIPE)

    if sys.version_info[0] < 3: 
        from StringIO import StringIO
    else:
        from io import StringIO
    
    b = StringIO(a.communicate()[0].decode('utf-8'))
    
    df = pd.read_csv(b,names=["track","time", "type", "channel","note","velocity", "other", "encoded"], sep=",")
    return df

In [3]:
def getQuarterNoteRate(df):
#     print(df)
    noteRate = df['velocity'].iloc[0]
    return noteRate

In [4]:
def setQuarterNoteTime(df, noteRate, sampleRate):
    df['time']=df['time'].apply(lambda x:int(math.floor(x/(noteRate/sampleRate))))
    return df

In [5]:
def removeControlC(df):
    indexNames = df[ df['type'].str.contains('Control_c')].index
    # Delete these row indexes from dataFrame
    out = df.drop(indexNames )
    #print(out)
    return out

In [6]:
def getNote(x):
    # make easier with: df.name.str.extract(r'([\d]+)',expand=False)
    arr = x.split('|')
    return arr[1]

def reformatStrings(df):
    df['note'] = df['note'].fillna(0).astype(int)
    df['encoded'] = df['type']+"|"+df['note'].astype(str)+"|"+df['velocity'].astype(str)
    df['finalEncode'] = df['encoded'].apply(lambda x: "off" + getNote(x)  if x.endswith("|0.0") else ("on" + getNote(x)))

    return df


In [7]:
def trackisOnlyPianolike(reformedRows):
    if (len(reformedRows[reformedRows['type'].str.contains('Program_c')]) == 1):
#         print(reformedRows.iloc[0])
        if (0<=int(reformedRows.iloc[0].channel)<8):
            return True
        else:
            print("not Piano Track")
            return False
    else:
        print("changes in program")
        return False


In [8]:
def makeTrueRows(reformedRows):
#     print(reformedRows['type'].nunique())
    trueRows = reformedRows[reformedRows['type'].str.contains('Note_on_c')]
    return trueRows

In [9]:
def makeNoteString(trueRows):
    startTime = trueRows['time'].iloc[0]
    prevTime = int(startTime)
    outArr = []
    for index, row in trueRows.iterrows():
        rowTime = int(row['time'])
#         print(row)
        if prevTime == rowTime:
            outArr.append(row['finalEncode'])
#             outPut += " " + row['finalEncode']
        elif prevTime < rowTime:
            diffTime = rowTime - 1 - prevTime
            outArr.append("wait"+str(diffTime))
#             print(row['finalEncode'])
            outArr.append(row['finalEncode'])
            prevTime = int(rowTime)
#             outPut += " " + "wait"+str(diffTime)
#             outPut += " " + row['finalEncode']

#     print(outArr)
    return  " ".join(outArr)
#     for i in range (indexes[2]+2,indexes[3]):
#         print( diffTime.loc[i])

In [10]:
def removeTitle(df):
    indexNames = df[ df['type'].str.contains('Title_t')].index
    # Delete these row indexes from dataFrame
    out = df.drop(indexNames )
    #print(out)
    return out

In [11]:
def trainingString(diffTime):
    diffTime.index = pd.RangeIndex(len(diffTime.index))
    search_values = ['Start_track','End_track']
    #TODO deal with multiple tracks?
    # diffTime.head(20)
    startStops = diffTime[diffTime.type.str.contains('|'.join(search_values ))]
    indexes = startStops.index
    reformedRows = diffTime.iloc[ indexes[2]+1:indexes[3] , :]
#     print(reformedRows)
    #make new df with only the note on note off ones
    if (trackisOnlyPianolike(reformedRows)):
        trueRows = makeTrueRows(reformedRows)
    #     print(trueRows.count())
        output = makeNoteString(trueRows)
        return output

In [12]:
def processFile(fileName):
    df = midiToCsv(fileName)
#     print(df.head(20))
    noteRate = getQuarterNoteRate(df)
    withoutC = removeControlC(df)
    withoutT = removeTitle(withoutC)
    diffTime = setQuarterNoteTime(withoutT, noteRate, 12)
#     diffTime.head(20)

    reformatStrings(diffTime)
    return trainingString(diffTime)

In [13]:
def splitData(musicPath):
    songList = pd.read_csv (musicPath + '/data/maestro/maestro-v2.0.0.csv')
#     print(songList.shape)
    train = songList[songList["split"] == 'train']
    test = songList[songList["split"] == 'test']
    validation = songList[songList["split"] == 'validation']

    return train, test, validation

In [14]:
train, test, validate = splitData(musicPath)

In [15]:
def encodeFile(subDirectory, fileName):
    filePath = subDirectory+fileName
    print(filePath)
    return processFile(filePath)
    

In [16]:
def cleanFileAndSave(row, folder):
    encodedString = encodeFile('/data/maestro/', row['midi_filename'])
    withEnding = row['midi_filename'].split("/")[1]
    name = withEnding.split(".")[0]+".txt"
    text_file = open(musicPath+'/data/cleanMaestro/'+folder+"/"+ name, "w")
    n = text_file.write(encodedString)
    text_file.close()

In [17]:
def processDF(dataFrame, folder):
    for index, row in dataFrame.iterrows():
        cleanFileAndSave(row, folder)

In [18]:
# train['midi_filename'].iloc[0]
# "2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi"

In [ ]:
# cleanFileAndSave(train.iloc[0], "train")

In [19]:
# processDF(train, "train")

In [20]:
def parallelProcessDF(dataFrame, folder):
    with concurrent.futures.ProcessPoolExecutor() as executor:
#         results = [executor.submit(cleanFileAndSave, row, folder) for index, row in dataFrame.iterrows()]
        for index, row in dataFrame.iterrows():
            executor.submit(cleanFileAndSave, row, folder)
#         for f in concurrent.futures.as_completed(results):
#             print(f.result())

In [23]:
parallelProcessDF(train, "train")

/data/maestro/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.midi
/data/maestro/2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.midi
/data/maestro/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi
/data/maestro/2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--3.midi
/data/maestro/2011/MIDI-Unprocessed_15_R1_2011_MID--AUDIO_R1-D6_09_Track09_wav.midi
/data/maestro/2006/MIDI-Unprocessed_19_R1_2006_01-07_ORIG_MID--AUDIO_19_R1_2006_04_Track04_wav.midi
/data/maestro/2013/ORIG-MIDI_02_7_10_13_Group_MID--AUDIO_11_R3_2013_wav--3.midi
/data/maestro/2011/MIDI-Unprocessed_09_R1_2011_MID--AUDIO_R1-D3_15_Track15_wav.midi
/data/maestro/2014/MIDI-UNPROCESSED_14-15_R1_2014_MID--AUDIO_15_R1_2014_wav--5.midi
/data/maestro/2008/MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--3.midi
/data/maestro/2006/MIDI-U

/data/maestro/2017/MIDI-Unprocessed_070_PIANO070_MID--AUDIO-split_07-08-17_Piano-e_1-02_wav--3.midi
/data/maestro/2017/MIDI-Unprocessed_072_PIANO072_MID--AUDIO-split_07-08-17_Piano-e_1-06_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_074_PIANO074_MID--AUDIO-split_07-08-17_Piano-e_2-04_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_081_PIANO081_MID--AUDIO-split_07-09-17_Piano-e_2_-02_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_083_PIANO083_MID--AUDIO-split_07-09-17_Piano-e_2_-06_wav--4.midi
/data/maestro/2008/MIDI-Unprocessed_08_R3_2008_01-05_ORIG_MID--AUDIO_08_R3_2008_wav--3.midi
/data/maestro/2008/MIDI-Unprocessed_10_R3_2008_01-05_ORIG_MID--AUDIO_10_R3_2008_wav--1.midi
/data/maestro/2008/MIDI-Unprocessed_11_R3_2008_01-04_ORIG_MID--AUDIO_11_R3_2008_wav--2.midi
/data/maestro/2015/MIDI-Unprocessed_R2_D1-2-3-6-7-8-11_mid--AUDIO-from_mp3_03_R2_2015_wav--2.midi
/data/maestro/2015/MIDI-Unprocessed_R2_D1-2-3-6-7-8-11_mid--AUDIO-from_mp3_07_R2_2015_wav--2.midi
/data/maestro/2013/ORIG-MI

/home/dmh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  


/data/maestro/2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_02_R1_2015_wav--6.midi
/data/maestro/2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_12_R1_2015_wav--5.midi
/data/maestro/2017/MIDI-Unprocessed_046_PIANO046_MID--AUDIO-split_07-06-17_Piano-e_2-02_wav--3.midi
/data/maestro/2017/MIDI-Unprocessed_054_PIANO054_MID--AUDIO-split_07-07-17_Piano-e_1-02_wav--3.midi
/data/maestro/2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--4.midi
/data/maestro/2015/MIDI-Unprocessed_R1_D2-21-22_mid--AUDIO-from_mp3_22_R1_2015_wav--5.midi
/data/maestro/2011/MIDI-Unprocessed_23_R2_2011_MID--AUDIO_R2-D6_04_Track04_wav.midi
/data/maestro/2017/MIDI-Unprocessed_043_PIANO043_MID--AUDIO-split_07-06-17_Piano-e_1-03_wav--4.midi
/data/maestro/2017/MIDI-Unprocessed_081_PIANO081_MID--AUDIO-split_07-09-17_Piano-e_2_-02_wav--4.midi
/data/maestro/2006/MIDI-Unprocessed_02_R1_2006_01-04_ORIG_MID--AUDIO_02_R1_2006_03_Track03_wav.midi
/data/maestro/2004/MIDI-Unprocessed_SMF_12_01

/data/maestro/2018/MIDI-Unprocessed_Recital9-11_MID--AUDIO_11_R1_2018_wav--1.midi
/data/maestro/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--1.midi
/data/maestro/2013/ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_04_R2_2013_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_079_PIANO079_MID--AUDIO-split_07-09-17_Piano-e_1-04_wav--2.midi
/data/maestro/2011/MIDI-Unprocessed_04_R2_2011_MID--AUDIO_R2-D2_02_Track02_wav.midi
/data/maestro/2011/MIDI-Unprocessed_25_R2_2011_MID--AUDIO_R2-D6_07_Track07_wav.midi
/data/maestro/2008/MIDI-Unprocessed_02_R2_2008_01-05_ORIG_MID--AUDIO_02_R2_2008_wav--1.midi
/data/maestro/2008/MIDI-Unprocessed_09_R2_2008_01-05_ORIG_MID--AUDIO_09_R2_2008_wav--1.midi
/data/maestro/2008/MIDI-Unprocessed_12_R2_2008_01-04_ORIG_MID--AUDIO_12_R2_2008_wav--2.midi
/data/maestro/2015/MIDI-Unprocessed_R2_D2-19-21-22_mid--AUDIO-from_mp3_22_R2_2015_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_070_PIANO070_MID--AUDIO-split_07-08-17_Piano-e_1-02_wav--1.midi
/data/ma

...

/data/maestro/2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_19_R1_2018_wav--1.midi
/data/maestro/2014/MIDI-UNPROCESSED_04-05_R1_2014_MID--AUDIO_05_R1_2014_wav--1.midi
/data/maestro/2015/MIDI-Unprocessed_R2_D2-19-21-22_mid--AUDIO-from_mp3_19_R2_2015_wav--1.midi
/data/maestro/2009/MIDI-Unprocessed_06_R1_2009_03-07_ORIG_MID--AUDIO_06_R1_2009_06_R1_2009_03_WAV.midi
/data/maestro/2009/MIDI-Unprocessed_06_R1_2009_03-07_ORIG_MID--AUDIO_06_R1_2009_06_R1_2009_04_WAV.midi
/data/maestro/2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_08_R1_2015_wav--1.midi
/data/maestro/2018/MIDI-Unprocessed_Recital5-7_MID--AUDIO_06_R1_2018_wav--1.midi
/data/maestro/2013/ORIG-MIDI_01_7_10_13_Group_MID--AUDIO_08_R3_2013_wav--2.midi
/data/maestro/2008/MIDI-Unprocessed_08_R1_2008_01-05_ORIG_MID--AUDIO_08_R1_2008_wav--1.midi
/data/maestro/2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi
/data/maestro/2004/MIDI-Unprocessed_SMF_22_R1_2004_01-04_ORIG_MID--AUDIO_22_R1_2004_03_

/data/maestro/2008/MIDI-Unprocessed_12_R1_2008_01-04_ORIG_MID--AUDIO_12_R1_2008_wav--2.midi
/data/maestro/2008/MIDI-Unprocessed_15_R1_2008_01-04_ORIG_MID--AUDIO_15_R1_2008_wav--1.midi
/data/maestro/2008/MIDI-Unprocessed_18_R1_2008_01-04_ORIG_MID--AUDIO_18_R1_2008_wav--1.midi
/data/maestro/2015/MIDI-Unprocessed_R1_D2-21-22_mid--AUDIO-from_mp3_21_R1_2015_wav--1.midi
/data/maestro/2013/ORIG-MIDI_02_7_6_13_Group__MID--AUDIO_07_R1_2013_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_057_PIANO057_MID--AUDIO-split_07-07-17_Piano-e_1-07_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_060_PIANO060_MID--AUDIO-split_07-07-17_Piano-e_2-04_wav--1.midi
/data/maestro/2008/MIDI-Unprocessed_04_R1_2008_01-04_ORIG_MID--AUDIO_04_R1_2008_wav--1.midi
/data/maestro/2011/MIDI-Unprocessed_20_R1_2011_MID--AUDIO_R1-D8_02_Track02_wav.midi
/data/maestro/2015/MIDI-Unprocessed_R1_D2-13-20_mid--AUDIO-from_mp3_15_R1_2015_wav--1.midi
/data/maestro/2011/MIDI-Unprocessed_22_R1_2011_MID--AUDIO_R1-D8_12_Track12_wav.midi
/d

/data/maestro/2014/MIDI-UNPROCESSED_14-15_R1_2014_MID--AUDIO_15_R1_2014_wav--1.midi
/data/maestro/2018/MIDI-Unprocessed_Chamber1_MID--AUDIO_07_R3_2018_wav--2.midi
/data/maestro/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--3.midi
/data/maestro/2011/MIDI-Unprocessed_19_R1_2011_MID--AUDIO_R1-D7_13_Track13_wav.midi
/data/maestro/2011/MIDI-Unprocessed_08_R1_2011_MID--AUDIO_R1-D3_08_Track08_wav.midi
/data/maestro/2011/MIDI-Unprocessed_15_R1_2011_MID--AUDIO_R1-D6_08_Track08_wav.midi
/data/maestro/2013/ORIG-MIDI_02_7_6_13_Group__MID--AUDIO_08_R1_2013_wav--2.midi
/data/maestro/2017/MIDI-Unprocessed_050_PIANO050_MID--AUDIO-split_07-06-17_Piano-e_3-01_wav--2.midi
/data/maestro/2017/MIDI-Unprocessed_053_PIANO053_MID--AUDIO-split_07-06-17_Piano-e_3-04_wav--3.midi
/data/maestro/2017/MIDI-Unprocessed_055_PIANO055_MID--AUDIO-split_07-07-17_Piano-e_1-04_wav--2.midi
/data/maestro/2017/MIDI-Unprocessed_057_PIANO057_MID--AUDIO-split_07-07-17_Piano-e_1-07_wav--2.midi
/data/maestro/2009/MIDI-Un

/data/maestro/2008/MIDI-Unprocessed_09_R3_2008_01-07_ORIG_MID--AUDIO_09_R3_2008_wav--4.midi
/data/maestro/2008/MIDI-Unprocessed_12_R3_2008_01-04_ORIG_MID--AUDIO_12_R3_2008_wav--1.midi
/data/maestro/2015/MIDI-Unprocessed_R2_D2-12-13-15_mid--AUDIO-from_mp3_12_R2_2015_wav--1.midi
/data/maestro/2015/MIDI-Unprocessed_R2_D2-19-21-22_mid--AUDIO-from_mp3_21_R2_2015_wav--2.midi
/data/maestro/2015/MIDI-Unprocessed_R2_D2-19-21-22_mid--AUDIO-from_mp3_22_R2_2015_wav--2.midi
/data/maestro/2013/ORIG-MIDI_01_7_10_13_Group_MID--AUDIO_07_R3_2013_wav--2.midi
/data/maestro/2013/ORIG-MIDI_01_7_10_13_Group_MID--AUDIO_08_R3_2013_wav--1.midi
/data/maestro/2013/ORIG-MIDI_02_7_10_13_Group_MID--AUDIO_12_R3_2013_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_071_PIANO071_MID--AUDIO-split_07-08-17_Piano-e_1-04_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_073_PIANO073_MID--AUDIO-split_07-08-17_Piano-e_2-02_wav--1.midi
/data/maestro/2017/MIDI-Unprocessed_075_PIANO075_MID--AUDIO-split_07-08-17_Piano-e_2-06_wav--2

/data/maestro/2008/MIDI-Unprocessed_15_R2_2008_01-04_ORIG_MID--AUDIO_15_R2_2008_wav--3.midi
/data/maestro/2004/MIDI-Unprocessed_XP_08_R1_2004_04-06_ORIG_MID--AUDIO_08_R1_2004_05_Track05_wav--2.midi
/data/maestro/2017/MIDI-Unprocessed_060_PIANO060_MID--AUDIO-split_07-07-17_Piano-e_2-04_wav--2.midi
/data/maestro/2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_03_R1_2018_wav--4.midi
/data/maestro/2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_03_R1_2018_wav--5.midi
/data/maestro/2013/ORIG-MIDI_01_7_6_13_Group__MID--AUDIO_02_R1_2013_wav--4.midi
/data/maestro/2006/MIDI-Unprocessed_22_R1_2006_01-04_ORIG_MID--AUDIO_22_R1_2006_04_Track04_wav.midi
/data/maestro/2011/MIDI-Unprocessed_17_R3_2011_MID--AUDIO_R3-D6_03_Track03_wav.midi
/data/maestro/2011/MIDI-Unprocessed_17_R3_2011_MID--AUDIO_R3-D6_04_Track04_wav.midi
/data/maestro/2011/MIDI-Unprocessed_17_R3_2011_MID--AUDIO_R3-D6_05_Track05_wav.midi
/data/maestro/2011/MIDI-Unprocessed_17_R3_2011_MID--AUDIO_R3-D6_06_Track06_wav.midi
/data/maestro/2011/MIDI-

In [24]:
parallelProcessDF(test, "test")

/data/maestro/2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.midi
/data/maestro/2006/MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_04_Track04_wav.midi
/data/maestro/2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_07_WAV.midi
/data/maestro/2006/MIDI-Unprocessed_24_R1_2006_01-05_ORIG_MID--AUDIO_24_R1_2006_03_Track03_wav.midi
/data/maestro/2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_08_WAV.midi
/data/maestro/2018/MIDI-Unprocessed_Recital13-15_MID--AUDIO_15_R1_2018_wav--1.midi
/data/maestro/2014/MIDI-UNPROCESSED_09-10_R1_2014_MID--AUDIO_09_R1_2014_wav--4.midi
/data/maestro/2009/MIDI-Unprocessed_11_R1_2009_01-05_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_04_WAV.midi
/data/maestro/2011/MIDI-Unprocessed_04_R3_2011_MID--AUDIO_R3-D2_05_Track05_wav.midi
/data/maestro/2004/MIDI-Unprocessed_SMF_17_R1_2004_03-06_ORIG_MID--AUDIO_20_R2_2004_12_Track12_wav--1.midi
/data/maestro/2008/MIDI-Unproc

/data/maestro/2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--4.midi
/data/maestro/2011/MIDI-Unprocessed_22_R2_2011_MID--AUDIO_R2-D5_11_Track11_wav.midi
/data/maestro/2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_02_Track02_wav.midi
/data/maestro/2008/MIDI-Unprocessed_12_R1_2008_01-04_ORIG_MID--AUDIO_12_R1_2008_wav--4.midi
/data/maestro/2009/MIDI-Unprocessed_01_R1_2009_01-04_ORIG_MID--AUDIO_01_R1_2009_01_R1_2009_04_WAV.midi
/data/maestro/2011/MIDI-Unprocessed_19_R1_2011_MID--AUDIO_R1-D7_15_Track15_wav.midi
/data/maestro/2009/MIDI-Unprocessed_07_R1_2009_01-03_ORIG_MID--AUDIO_07_R1_2009_07_R1_2009_03_WAV.midi
/data/maestro/2013/ORIG-MIDI_01_7_6_13_Group__MID--AUDIO_04_R1_2013_wav--4.midi
/data/maestro/2018/MIDI-Unprocessed_Recital5-7_MID--AUDIO_06_R1_2018_wav--3.midi
/data/maestro/2013/ORIG-MIDI_02_7_10_13_Group_MID--AUDIO_14_R3_2013_wav--2.midi
/data/maestro/2015/MIDI-Unprocessed_R1_D2-13-20_mid--AUDIO-from_mp3_15_R1_2015_wav--4.midi
/data/maestro/2013/ORI

In [25]:
parallelProcessDF(validate, "valid")

/data/maestro/2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.midi
/data/maestro/2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_05_WAV.midi
/data/maestro/2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AUDIO_07_R1_2009_07_R1_2009_04_WAV.midi
/data/maestro/2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_06_WAV.midi
/data/maestro/2013/ORIG-MIDI_01_7_6_13_Group__MID--AUDIO_03_R1_2013_wav--4.midi
/data/maestro/2014/MIDI-UNPROCESSED_14-15_R1_2014_MID--AUDIO_15_R1_2014_wav--3.midi
/data/maestro/2014/MIDI-UNPROCESSED_14-15_R1_2014_MID--AUDIO_15_R1_2014_wav--4.midi
/data/maestro/2006/MIDI-Unprocessed_22_R1_2006_01-04_ORIG_MID--AUDIO_22_R1_2006_02_Track02_wav.midi
/data/maestro/2018/MIDI-Unprocessed_Recital8_MID--AUDIO_08_R1_2018_wav--2.midi
/data/maestro/2018/MIDI-Unprocessed_Recital9-11_MID--AUDIO_10_R1_2018_wav--5.midi
/data/maestro/2008/MIDI-Unprocessed_02_R1_2008_01-05_ORIG_MID--AUDIO_02_R1_2008_wa

/data/maestro/2006/MIDI-Unprocessed_23_R1_2006_01-05_ORIG_MID--AUDIO_23_R1_2006_02_Track02_wav.midi
/data/maestro/2011/MIDI-Unprocessed_03_R3_2011_MID--AUDIO_R3-D1_03_Track03_wav.midi
/data/maestro/2004/MIDI-Unprocessed_XP_19_R1_2004_01-02_ORIG_MID--AUDIO_19_R1_2004_01_Track01_wav.midi
/data/maestro/2004/MIDI-Unprocessed_SMF_13_01_2004_01-05_ORIG_MID--AUDIO_13_R1_2004_05_Track05_wav.midi
/data/maestro/2018/MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1.midi
/data/maestro/2004/MIDI-Unprocessed_XP_10_R1_2004_01-02_ORIG_MID--AUDIO_10_R1_2004_02_Track02_wav.midi
/data/maestro/2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_03_R1_2015_wav--3.midi
/data/maestro/2015/MIDI-Unprocessed_R1_D2-13-20_mid--AUDIO-from_mp3_13_R1_2015_wav--2.midi
/data/maestro/2014/MIDI-UNPROCESSED_16-18_R1_2014_MID--AUDIO_16_R1_2014_wav--2.midi
/data/maestro/2017/MIDI-Unprocessed_058_PIANO058_MID--AUDIO-split_07-07-17_Piano-e_2-02_wav--3.midi
/data/maestro/2018/MIDI-Unprocessed_Recital4_MID--AUDIO_04_R1_2

/home/dmh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  


/data/maestro/2006/MIDI-Unprocessed_13_R1_2006_01-06_ORIG_MID--AUDIO_13_R1_2006_05_Track05_wav.midi
/data/maestro/2011/MIDI-Unprocessed_21_R1_2011_MID--AUDIO_R1-D8_10_Track10_wav.midi
/data/maestro/2014/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--5.midi
/data/maestro/2013/ORIG-MIDI_03_7_10_13_Group_MID--AUDIO_15_R3_2013_wav--2.midi
/data/maestro/2004/MIDI-Unprocessed_XP_19_R1_2004_01-02_ORIG_MID--AUDIO_19_R1_2004_03_Track03_wav.midi
/data/maestro/2014/MIDI-UNPROCESSED_19-20_R1_2014_MID--AUDIO_20_R1_2014_wav--2.midi
/data/maestro/2011/MIDI-Unprocessed_23_R3_2011_MID--AUDIO_R3-D8_03_Track03_wav.midi
/data/maestro/2018/MIDI-Unprocessed_Recital4_MID--AUDIO_04_R1_2018_wav--4.midi
/data/maestro/2014/MIDI-UNPROCESSED_04-05_R1_2014_MID--AUDIO_04_R1_2014_wav--3.midi
/data/maestro/2017/MIDI-Unprocessed_073_PIANO073_MID--AUDIO-split_07-08-17_Piano-e_2-02_wav--4.midi
/data/maestro/2011/MIDI-Unprocessed_12_R2_2011_MID--AUDIO_R2-D4_02_Track02_wav.midi
/data/maestro/2006/MIDI-Unprocesse

In [ ]:
# processFile("/data/maestro/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi")

### Encode DF to MIDI

In [ ]:
df = midiToCsv("/data/maestro/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi")

In [ ]:
def dfToCSV(file_name, df):
    df.to_csv(dataPath +  file_name, index=False, header=False)

In [ ]:
def dfToMIDI(file_name,df,out_file):
    dfToCSV("/misc/simpleCSV",df)
    cmd = ['./csvmidi', '-v', dataPath+file_name, musicPath + out_file]
    a = subprocess.Popen(cmd, stdout=subprocess.PIPE)

    if sys.version_info[0] < 3: 
        from StringIO import StringIO
    else:
        from io import StringIO

#     b = StringIO(a.communicate()[0].decode('utf-8'))
#     df = pd.read_csv(b,names=["track","time", "type", "channel","note","velocity", "other", "encoded"], sep=",")
#     return df
    

In [ ]:
def removeEverything(df):
    indexNames = df[ df['type'].str.contains('Control_c')].index
    # Delete these row indexes from dataFrame
    df = df.drop(indexNames )
    indexNames = df[ df['type'].str.contains('Title_t')].index
    # Delete these row indexes from dataFrame
    df = df.drop(indexNames )
    indexNames = df[ df['type'].str.contains('Note_on_c')].index
    # Delete these row indexes from dataFrame
    df = df.drop(indexNames )
    df = df.drop(columns=['other', 'encoded'])
    df = df.replace(np.nan,0)
    df = df.reset_index(drop = True)
    return df

In [ ]:
def readOutput(filePath):
    with open(musicPath + filePath, 'r') as file:
        data = file.read()
#     s = file.read()
#     file.close()
    return data

In [ ]:
def getWaitTime(wait):
    time = wait.replace("wait","")
    return int(time)

In [ ]:
def getOnValue(val):
    num = val.replace("on","")
    return int(num)

In [ ]:
def getOffValue(val):
    num = val.replace("off","")
    return int(num)

In [ ]:
def stringToPandasDF(s):
    arr = s.split()
    notes = []
    times = []
    velocities = []
    onOff = []
    track = []
    channel = []
    time = 0
    for val in arr:
        if "wait" in val:
            t = getWaitTime(val)
            time += t+1
#             print(t)
        elif "on" in val:
#             print(time)
            num = getOnValue(val)
#             print(val)
            times.append(time*(384/12))
            velocities.append(81)
            onOff.append("Note_on_c")
            notes.append(num)
            track.append(2)
            channel.append(0)
        elif "off" in val:
            num = getOffValue(val)
            velocities.append(0)
            onOff.append("Note_on_c")
            times.append(time*(384/12))
            #384 is the division value from the header, in other words the number of time per quartenote. 
            #12 is the samplerate used at this moment
            track.append(2)
            channel.append(0)
            notes.append(num)
#     print(arr)
#     out = pd.Series(arr)
#     print(out)
    df = pd.DataFrame(#TODO
    { 
        'track': track,
     'time': times,
     'type': onOff,
     'channel': channel,
     'note': notes,
     'velocity': velocities
    })
    return df

In [ ]:
out = removeEverything(df)

In [ ]:
out

In [ ]:
string = readOutput("/data/cleanMaestro/train/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.txt")

In [ ]:
rows = stringToPandasDF(string)

out.at[7,"time"] = rows['time'].iloc[-1]

In [ ]:
start = out.iloc[:7,:]
end = out.iloc[7:,:]

In [ ]:
endDF= pd.concat([start, rows,end],  sort=False).reset_index(drop=True)

In [ ]:
dfToMIDI("/misc/simpleCSV",out,"/output/firstTry.midi" )

In [ ]:
dfToMIDI("/misc/simpleCSV",endDF,"/output/test.midi" )